# COMPARISION OF TTS MODELS FOR LATENCY

# AI4BHARAT - PARLER

In [1]:
pip install git+https://github.com/huggingface/parler-tts.git


  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-7f8qagly
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-7f8qagly
  Resolved https://github.com/huggingface/parler-tts.git to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-64x1sez7/descript-audiotools_115bb276f2a84af0a6839e5638a3166b
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-64x1sez7/descript-audiotools_115bb276f2a84af0a6839e5638a3166b
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.9 MB/s eta 0:00:00
  Prep

In [2]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.75G [00:00<?, ?B/s]

  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab_size": 32128
}

  "_name_or_path": "ylacombe/dac_44khz",
  "architectures": [
    "DacModel"
  ],
  "codebook_dim": 8,
  "codebook_loss_weight": 1.0,
  "codebook_size": 1024,
  "commitment_loss_weight": 0.25,
  "decoder_hidden_si

generation_config.json:   0%|          | 0.00/223 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:
prompt = "இந்த உலகம் மிகவும் அழகானது, இதில் நாம் ஒருவருக்கொருவர் உதவிசெய்து வாழ்ந்தால், வாழ்க்கை மேலும் அர்த்தமுள்ளதாகும்."
description = "A calm, warm female voice with medium pitch and a steady speed."


description_input_ids = description_tokenizer(description, return_tensors="pt").to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").to(device)

start_time = time.time()

generation = model.generate(input_ids=description_input_ids.input_ids, attention_mask=description_input_ids.attention_mask, prompt_input_ids=prompt_input_ids.input_ids, prompt_attention_mask=prompt_input_ids.attention_mask)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("indic_tts_out.wav", audio_arr, model.config.sampling_rate)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")


✅ Generation time: 22.39 seconds


## SARVAM AI TTS

# GPT

In [2]:
from pathlib import Path
from openai import OpenAI

client = OpenAI()

speech_file_path = "content/speech.mp3"

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="Today is a wonderful day to build",
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file_path)

ModuleNotFoundError: No module named 'openai'

In [ ]:
import time
import openai

for attempt in range(5):
    try:
        # your OpenAI API call here
        break
    except openai.RateLimitError:
        wait = 2 ** attempt
        print(f"Rate limit hit. Waiting {wait} seconds...")
        time.sleep(wait)


# GEMINI

In [24]:
pip install google-cloud-texttospeech


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 28.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.7
    Uninstalling protobuf-4.25.7:
      Successfully uninstalled protobuf-4.25.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
descript-audiotools 0.7.4 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.29.4 which is incompatible.


In [8]:
!pip install git+https://github.com/facebookresearch/seamless_communication.git
!pip install torch torchaudio

  Cloning https://github.com/facebookresearch/seamless_communication.git to /tmp/pip-req-build-jizjkyhy
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/seamless_communication.git /tmp/pip-req-build-jizjkyhy
  Resolved https://github.com/facebookresearch/seamless_communication.git to commit 90e2b57ac4d82fa2bfaa25caeffe39ceb8b2ebec
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 41.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s 

In [3]:
from seamless_communication.models.inference import Translator


# Initialize the translator
translator = Translator("seamlessM4T_large")

# Translate speech-to-speech
result = translator.translate(
    input_audio="harvard.wav",  # Your input audio file
    src_lang="eng",                   # Source language code
    tgt_lang="hin",                   # Target language code
    task="s2st"                       # Speech-to-speech translation
)

# Save output audio
with open("output_speech.wav", "wb") as f:
    f.write(result["audio"])
print("Translation complete! Output saved to output_speech.wav")


ModuleNotFoundError: No module named 'seamless_communication.models.inference'

In [4]:
from transformers import SeamlessM4TForSpeechToSpeech

RuntimeError: Failed to import transformers.models.seamless_m4t.modeling_seamless_m4t because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
type object 'torch._C.Tag' has no attribute 'needs_fixed_stride_order'

In [5]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)


2.2.2+cu121
4.46.1


# google translator

In [7]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=cff48375b79a4efb0a208db15f0f272f4fca7690dd8afb361e9ae76d4177edb7
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [14]:
from googletrans import Translator, LANGUAGES
import time
translator = Translator()


# Example: Translate from English to Hindi
text = """बिल्कुल! यहाँ एक लगभग 1000 शब्दों का हिंदी गद्यांश है, जिसमें भारतीय संस्कृति, भाषा, शिक्षा, विज्ञान, समाज, और पर्यावरण जैसे विषयों को समाहित किया गया है। यह टेक्स्ट आपके किसी भी NLP, TTS, या अनुवाद मॉडल के लिए आदर्श है।

भारत एक विविधताओं से भरा देश है, जहाँ विभिन्न भाषाएँ, धर्म, संस्कृतियाँ और परंपराएँ एक साथ मिलकर एक अद्वितीय समाज का निर्माण करती हैं। भारतीय संस्कृति की जड़ें हजारों वर्षों पुरानी हैं और यह आज भी अपनी प्राचीनता और आधुनिकता का सुंदर संगम प्रस्तुत करती है। यहाँ की भाषाएँ, साहित्य, कला, संगीत, नृत्य, वास्तुकला, और विज्ञान ने न केवल भारत को बल्कि पूरे विश्व को समृद्ध किया है।

भारतीय भाषाओं की बात करें तो हिंदी, जो कि भारत की राजभाषा है, करोड़ों लोगों द्वारा बोली और समझी जाती है। हिंदी साहित्य का इतिहास भी अत्यंत समृद्ध है। कबीर, तुलसीदास, सूरदास, प्रेमचंद, महादेवी वर्मा, हरिवंश राय बच्चन जैसे कवि और लेखक हिंदी साहित्य को नई ऊँचाइयों तक ले गए। हिंदी भाषा ने न केवल साहित्य में, बल्कि सिनेमा, रेडियो, टेलीविजन और इंटरनेट के माध्यम से भी जनमानस में अपनी गहरी छाप छोड़ी है।

भारत में शिक्षा का भी एक गौरवशाली इतिहास रहा है। प्राचीन काल में तक्षशिला, नालंदा, विक्रमशिला जैसे विश्वविद्यालयों में देश-विदेश से विद्यार्थी अध्ययन के लिए आते थे। आज भी भारत में शिक्षा को अत्यधिक महत्व दिया जाता है। यहाँ की शिक्षा प्रणाली में समय-समय पर सुधार होते रहे हैं, जिससे विद्यार्थियों को आधुनिक विज्ञान, तकनीक, गणित, चिकित्सा, और अन्य विषयों में उत्कृष्ट ज्ञान प्राप्त हो सके। डिजिटल शिक्षा और ऑनलाइन लर्निंग ने शिक्षा के क्षेत्र में क्रांति ला दी है, जिससे दूरदराज के क्षेत्रों में भी ज्ञान की पहुँच संभव हुई है।

भारतीय समाज में परिवार, रिश्ते, और सामाजिक मूल्यों का विशेष स्थान है। यहाँ संयुक्त परिवार की परंपरा रही है, जिसमें एक ही छत के नीचे कई पीढ़ियाँ साथ रहती हैं। हालाँकि, समय के साथ समाज में बदलाव आया है और अब एकल परिवारों की संख्या बढ़ रही है, लेकिन फिर भी भारतीय समाज में आपसी सहयोग, सम्मान, और सहिष्णुता की भावना बनी हुई है। त्योहारों का भी यहाँ विशेष महत्व है। दीपावली, होली, ईद, क्रिसमस, गुरुपर्व, पोंगल, ओणम आदि त्योहार पूरे देश में हर्षोल्लास के साथ मनाए जाते हैं, जो सामाजिक एकता और सांस्कृतिक विविधता का प्रतीक हैं।

भारतीय विज्ञान और तकनीक ने भी विश्व में अपनी पहचान बनाई है। आर्यभट्ट, भास्कराचार्य, चरक, सुश्रुत जैसे वैज्ञानिकों और चिकित्सकों ने प्राचीन भारत में गणित, ज्योतिष, चिकित्सा, और सर्जरी के क्षेत्र में अद्वितीय योगदान दिया। आधुनिक भारत में भी इसरो, डीआरडीओ, आईआईटी, आईआईएससी जैसी संस्थाएँ विज्ञान और तकनीक के क्षेत्र में उत्कृष्ट कार्य कर रही हैं। भारत का चंद्रयान और मंगलयान मिशन, डिजिटल इंडिया, स्टार्टअप इंडिया जैसी पहलें देश को तकनीकी दृष्टि से सशक्त बना रही हैं।

पर्यावरण संरक्षण भी आज के समय की एक महत्वपूर्ण आवश्यकता है। भारत में वनों, नदियों, पर्वतों, और जैव विविधता का अनमोल खजाना है। लेकिन बढ़ती आबादी, शहरीकरण, और औद्योगीकरण के कारण पर्यावरण पर दबाव बढ़ा है। जलवायु परिवर्तन, वायु और जल प्रदूषण, वन्य जीवों की घटती संख्या जैसी समस्याएँ सामने आ रही हैं। ऐसे में, हमें पर्यावरण संरक्षण के लिए जागरूक होना होगा। वृक्षारोपण, जल संरक्षण, प्लास्टिक का कम उपयोग, और स्वच्छता जैसे कार्यों में सभी को भागीदारी निभानी चाहिए।

भारतीय युवाओं में अपार ऊर्जा, उत्साह और प्रतिभा है। वे शिक्षा, खेल, विज्ञान, कला, संगीत, साहित्य, और उद्यमिता के क्षेत्र में देश का नाम रोशन कर रहे हैं। आज के युवा डिजिटल युग के साथ कदम से कदम मिलाकर चल रहे हैं। सोशल मीडिया, इंटरनेट, और मोबाइल तकनीक ने उनकी सोच और दृष्टिकोण को वैश्विक बना दिया है। वे न केवल अपने देश की समस्याओं को समझते हैं, बल्कि उनके समाधान के लिए भी तत्पर रहते हैं।

भारतीय लोकतंत्र भी विश्व में अपनी एक अलग पहचान रखता है। यहाँ की संसद, न्यायपालिका, और कार्यपालिका संविधान के अनुसार कार्य करती हैं। नागरिकों को अभिव्यक्ति की स्वतंत्रता, शिक्षा का अधिकार, समानता, और न्याय जैसे मौलिक अधिकार प्राप्त हैं। चुनावों के माध्यम से जनता अपनी सरकार चुनती है, जिससे लोकतंत्र की जड़ें और मजबूत होती हैं।

अंत में, यह कहना उचित होगा कि भारत की विविधता, संस्कृति, भाषा, विज्ञान, और समाज ने उसे विश्व में एक विशिष्ट स्थान दिलाया है। यहाँ के लोग अपने मूल्यों, परंपराओं, और आधुनिक सोच के साथ आगे बढ़ रहे हैं। हमें अपने देश की इस अनमोल विरासत को संजोकर रखना चाहिए और आने वाली पीढ़ियों को भी इसके महत्व से अवगत कराना चाहिए। इसी में भारत की सच्ची प्रगति और समृद्धि निहित है।

अगर आपको किसी विशेष विषय या शैली में हिंदी टेक्स्ट चाहिए तो कृपया बताएं!"""

start_time = time.time()

translated = translator.translate(text, dest='en')

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")

print(f"Original: {text}")



Generation time: 1.25 seconds
Original: बिल्कुल! यहाँ एक लगभग 1000 शब्दों का हिंदी गद्यांश है, जिसमें भारतीय संस्कृति, भाषा, शिक्षा, विज्ञान, समाज, और पर्यावरण जैसे विषयों को समाहित किया गया है। यह टेक्स्ट आपके किसी भी NLP, TTS, या अनुवाद मॉडल के लिए आदर्श है।

भारत एक विविधताओं से भरा देश है, जहाँ विभिन्न भाषाएँ, धर्म, संस्कृतियाँ और परंपराएँ एक साथ मिलकर एक अद्वितीय समाज का निर्माण करती हैं। भारतीय संस्कृति की जड़ें हजारों वर्षों पुरानी हैं और यह आज भी अपनी प्राचीनता और आधुनिकता का सुंदर संगम प्रस्तुत करती है। यहाँ की भाषाएँ, साहित्य, कला, संगीत, नृत्य, वास्तुकला, और विज्ञान ने न केवल भारत को बल्कि पूरे विश्व को समृद्ध किया है।

भारतीय भाषाओं की बात करें तो हिंदी, जो कि भारत की राजभाषा है, करोड़ों लोगों द्वारा बोली और समझी जाती है। हिंदी साहित्य का इतिहास भी अत्यंत समृद्ध है। कबीर, तुलसीदास, सूरदास, प्रेमचंद, महादेवी वर्मा, हरिवंश राय बच्चन जैसे कवि और लेखक हिंदी साहित्य को नई ऊँचाइयों तक ले गए। हिंदी भाषा ने न केवल साहित्य में, बल्कि सिनेमा, रेडियो, टेलीविजन और इंटरनेट के माध्यम से 

In [12]:
print(f"Translated: {translated.text}")

Translated: Absolutely!There is a Hindi passage of about 1000 words, which contains subjects like Indian culture, language, education, science, society, and environment.This text is ideal for any of your NLP, TTS, or translation model.

India is a country full of diversity, where various languages, religion, cultures and traditions together form a unique society.The roots of Indian culture are thousands of years old and it still presents a beautiful confluence of its antiquity and modernity.The languages, literature, art, music, dance, architecture, and science here have enriched not only India but the whole world.

Talking about Indian languages, Hindi, which is the official language of India, is spoken and understood by crores of people.The history of Hindi literature is also very rich.Poets and writers like Kabir, Tulsidas, Surdas, Premchand, Mahadevi Verma, Harivansh Rai Bachchan took Hindi literature to new heights.Hindi language has left a deep impression in the public not only i

# ESPNET

In [5]:
!pip install espnet
!pip install espnet_model_zoo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.7/524.7 kB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 26.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
from espnet2.bin.s2st_inference import Speech2Speech

s2st = Speech2Speech.from_pretrained("espnet/your_s2st_model")
wav = s2st("harvard.wav")["wav"]
sf.write("output_s2s.wav", wav.numpy(), 22050)

# SMALLEST AI

In [22]:
text[:500]

'बिल्कुल! यहाँ एक लगभग 1000 शब्दों का हिंदी गद्यांश है, जिसमें भारतीय संस्कृति, भाषा, शिक्षा, विज्ञान, समाज, और पर्यावरण जैसे विषयों को समाहित किया गया है। यह टेक्स्ट आपके किसी भी NLP, TTS, या अनुवाद मॉडल के लिए आदर्श है।\n\nभारत एक विविधताओं से भरा देश है, जहाँ विभिन्न भाषाएँ, धर्म, संस्कृतियाँ और परंपराएँ एक साथ मिलकर एक अद्वितीय समाज का निर्माण करती हैं। भारतीय संस्कृति की जड़ें हजारों वर्षों पुरानी हैं और यह आज भी अपनी प्राचीनता और आधुनिकता का सुंदर संगम प्रस्तुत करती है। यहाँ की भाषाएँ, साहित्'

In [3]:
import requests



url = "https://waves-api.smallest.ai/api/v1/lightning/get_speech"

payload = {
    "voice_id": "emily",
    "text": text[200:450],
    "sample_rate": 24000,
    "add_wav_header": True
}

headers = {
    "Authorization": f"Bearer {}",
    "Content-Type": "application/json"
}


NameError: name 'text' is not defined

In [33]:
response = requests.request("POST", url, json=payload, headers=headers)

start_time = time.time()
if response.status_code == 200:
    with open("output.wav", "wb") as audio_file:
        audio_file.write(response.content)
else:
  print(response.status_code)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")

Generation time: 0.00 seconds


# SPEECH TO TEXT FOR LOW LATENCY

## SARVAM AI

In [3]:
import requests
import io

In [ ]:

api_url = "https://api.sarvam.ai/speech-to-text-translate"

headers = {"api-subscription-key": }

data = {
    "model": "saaras:v2",
    "with_diarization": False,
}

In [5]:

from pydub import AudioSegment


def split_audio(audio_path, chunk_duration_ms):
    """
    Splits an audio file into smaller chunks of specified duration.

    Args:
        audio_path (str): Path to the audio file to be split.
        chunk_duration_ms (int): Duration of each chunk in milliseconds.

    Returns:
        list: A list of AudioSegment objects representing the audio chunks.
    """
    audio = AudioSegment.from_file(audio_path)
    chunks = []
    if len(audio) > chunk_duration_ms:

        for i in range(0, len(audio), chunk_duration_ms):
            chunks.append(audio[i : i + chunk_duration_ms])
    else:

        chunks.append(audio)
    return chunks

In [6]:
def translate_audio(
    audio_file_path, api_url, headers, data, chunk_duration_ms=30 * 1000
):
    """
    Translates audio into text with optional diarization and timestamps.

    Args:
        audio_file_path (str): Path to the audio file.
        api_url (str): API endpoint URL for Speech-to-Text and Translate.
        headers (dict): Headers for API authentication.
        data (dict): Payload containing model and other options like diarization.
        chunk_duration_ms (int): Duration of each audio chunk in milliseconds.

    Returns:
        dict: Collated response containing the transcript and word-level timestamps.
    """
    chunks = split_audio(audio_file_path, chunk_duration_ms)
    responses = []


    for idx, chunk in enumerate(chunks):
        # Export the chunk to a BytesIO object (in-memory binary stream)
        chunk_buffer = io.BytesIO()
        chunk.export(chunk_buffer, format="wav")
        chunk_buffer.seek(0)  # Reset the pointer to the start of the stream

        # Prepare the file for the API request
        files = {"file": ("audiofile.wav", chunk_buffer, "audio/wav")}

        try:
            # Make the POST request to the API
            response = requests.post(api_url, headers=headers, files=files, data=data)
            if response.status_code == 200 or response.status_code == 201:
                print(f"Chunk {idx} POST Request Successful!")
                response_data = response.json()
                transcript = response_data.get("transcript", "")
                responses.append({"transcript": transcript})
            else:
                # Handle failed requests
                print(
                    f"Chunk {idx} POST Request failed with status code: {response.status_code}"
                )
                print("Response:", response.text)
        except Exception as e:
            # Handle any exceptions during the request
            print(f"Error processing chunk {idx}: {e}")
        finally:
            # Ensure the buffer is closed after processing
            chunk_buffer.close()

    # Collate the transcriptions from all chunks
    collated_transcript = " ".join([resp["transcript"] for resp in responses])
    collated_response = {
        "transcript": collated_transcript,
        "language": response_data.get("language_code"),
    }
    return collated_response


In [7]:
audio_file_path="/content/New-Recording.wav"

start_time=time.time()

translation = translate_audio(audio_file_path, api_url, headers, data)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")

translation

Chunk 0 POST Request Successful!
Chunk 1 POST Request Successful!
Chunk 2 POST Request Successful!
Chunk 3 POST Request Successful!
Generation time: 7.29 seconds


{'transcript': "Hi Vandita, how are you? Hi sister, how are you? Are you doing well? I am super. How is everyone at home? Hmm, they are doing well. How is everyone at your home? They are doing great. Also, I am currently running a retail business. Do you know anything about retail business? No, I called you to interview you about it. So, you... Ma'am, how did you become such a successful businesswoman? Can you explain? Okay ma'am, I'll tell you. First, how did I get this plan? Recently, about six months ago, I went to my friend's shop. Okay? She liked me a lot when she saw her shop. So, I have a mutation course. Okay? So, I really wanted to do it. So, I got motivated and applied. Yes, you are starting a retail business in a boutique, right? How do you manage to be so successful? Also, share your secrets, tips, experience, and knowledge. If you share everything a bit, it will be ready as a book. If you tell me, it will be useful for me. Okay, first, all the top five in India are in that

## WISHPHER

In [68]:
!pip install openai-whisper
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [73]:
import whisper

model = whisper.load_model("medium")


start_time=time.time()

result = model.transcribe("/content/New-Recording.wav", task="translate")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")


print("Detected language:", result["language"])
print("English transcript:", result["text"])


Generation time: 13.70 seconds
Detected language: ta
English transcript:  Hi Nivaitha, how are you? Hi sister, how are you? I am fine. Am I doing good? How are my parents? I am fine. How are my parents? They are doing good. I am running a retail business. Do you know anything about retail business? No, I didn't ask you about it. I called you myself. How did you become a successful businesswoman? I will tell you. I got this plan. I went to my friend's shop 6 months ago. I liked her shop a lot. I am also doing a job. I also wanted to do the same. I got motivated and opened my own business. You are running a business by selling boutique. How did you become successful? If you share your secrets, experience and knowledge, it will be useful.


# SEAMLESSM4

In [17]:
import torchaudio
import numpy as np


file_path = "/content/New-Recording.wav"
audio, orig_sample_rate = torchaudio.load(file_path)

if orig_sample_rate != 16000:
    audio = torchaudio.functional.resample(audio, orig_freq=orig_sample_rate, new_freq=16000)

audio_np = audio.squeeze().numpy()
sample_rate = 16000


In [16]:
from transformers import AutoProcessor, SeamlessM4TModel
import torch
import soundfile as sf
import time

processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")
model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium")

In [19]:
start_time=time.time()

inputs = processor(audios=audio, return_tensors="pt", sampling_rate=sample_rate)

output_tokens = model.generate(**inputs, tgt_lang="eng", generate_speech=False)

tokens = output_tokens.sequences[0]

english_text = processor.decode(tokens, skip_special_tokens=True)
print("English transcript:", english_text)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Generation time: {elapsed_time:.2f} seconds")

KeyboardInterrupt: 